In [1]:
import re
import os
import pandas as pd
import numpy as np

In [2]:
path_read ="/Users/a06411/Documents/data_hub/lending_club/lending_club.csv"

In [3]:
df = pd.read_csv(path_read)

/var/folders/h4/44486vyn1_xbs13g1z50l5m00000gn/T/ipykernel_18379/462175026.py:1: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path_read)


In [4]:
df.shape

(2260701, 151)

In [5]:
df['loan_status'].value_counts()

Fully Paid                                             1076751
Current                                                 878317
Charged Off                                             268559
Late (31-120 days)                                       21467
In Grace Period                                           8436
Late (16-30 days)                                         4349
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     40
Name: loan_status, dtype: int64

In [6]:
df = df[df['loan_status'] != 'Current']

In [7]:
df['loan_status'].value_counts()

Fully Paid                                             1076751
Charged Off                                             268559
Late (31-120 days)                                       21467
In Grace Period                                           8436
Late (16-30 days)                                         4349
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     40
Name: loan_status, dtype: int64

## 결측치 제거 

In [8]:
drop_cols = df.columns[df.isnull().mean() > 0.9]

In [9]:
drop_cols

Index(['member_id', 'desc', 'next_pymnt_d', 'annual_inc_joint', 'dti_joint',
       'verification_status_joint', 'revol_bal_joint',
       'sec_app_fico_range_low', 'sec_app_fico_range_high',
       'sec_app_earliest_cr_line', 'sec_app_inq_last_6mths',
       'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_revol_util',
       'sec_app_open_act_il', 'sec_app_num_rev_accts',
       'sec_app_chargeoff_within_12_mths',
       'sec_app_collections_12_mths_ex_med',
       'sec_app_mths_since_last_major_derog', 'hardship_type',
       'hardship_reason', 'hardship_status', 'deferral_term',
       'hardship_amount', 'hardship_start_date', 'hardship_end_date',
       'payment_plan_start_date', 'hardship_length', 'hardship_dpd',
       'hardship_loan_status', 'orig_projected_additional_accrued_interest',
       'hardship_payoff_balance_amount', 'hardship_last_payment_amount',
       'debt_settlement_flag_date', 'settlement_status', 'settlement_date',
       'settlement_amount', 'settlement_perce

In [10]:
df = df.drop(drop_cols,axis=1)

In [11]:
df.shape

(1382384, 112)

## 변수 제거

### 날짜변수 제거 

In [12]:
df = df.drop(['last_pymnt_d','last_credit_pull_d','earliest_cr_line'] , 1)

/var/folders/h4/44486vyn1_xbs13g1z50l5m00000gn/T/ipykernel_18379/773873711.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(['last_pymnt_d','last_credit_pull_d','earliest_cr_line'] , 1)


In [13]:
df.shape

(1382384, 109)

### 명칭 변수 제거 

In [14]:
df = df.drop(['url','emp_title','title','zip_code','policy_code'] , 1) 

/var/folders/h4/44486vyn1_xbs13g1z50l5m00000gn/T/ipykernel_18379/4087905089.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(['url','emp_title','title','zip_code','policy_code'] , 1)


In [15]:
df.shape

(1382384, 104)

## 등급 평가 없는 데이터 삭제 

In [17]:
df['grade'].isna().sum()

33

In [18]:
df[df['grade'].isna()].shape

(33, 104)

In [19]:
df[df['grade'].isna()].head(3)

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
421095,Total amount funded in policy code 1: 6417608175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421096,Total amount funded in policy code 2: 1944088810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
528961,Total amount funded in policy code 1: 1741781700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
df['grade'].notna().sum()

1382351

In [20]:
df = df[df['grade'].notna()]

In [22]:
df.shape

(1382351, 104)

In [23]:
df['verification_status'] = df.verification_status.map({'Verified': 0, 'Source Verified': 1, 'Not Verified': 2})
df['debt_settlement_flag'] = df.debt_settlement_flag.map({'N': 0, 'Y': 1})
df['initial_list_status'] = df.initial_list_status.map({'w': 0, 'f': 1})
df['application_type'] = df.application_type.map({'Individual': 0, 'Joint App': 1})
df['hardship_flag'] = df.hardship_flag.map({'N': 0, 'Y': 1})
df['pymnt_plan'] = df.pymnt_plan.map({'n': 0, 'y': 1})
df['disbursement_method'] = df.disbursement_method.map({'Cash': 0, 'DirectPay': 1})
df['grade'] = df.grade.map({'A': 0, 'B': 1,'C': 2, 'D': 3,'E': 4, 'F': 5,'G': 6}) 

In [24]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

df['addr_state'] = le.fit_transform(df['addr_state'].astype(str))
df['sub_grade'] = le.fit_transform(df['sub_grade'].astype(str))
df['purpose'] = le.fit_transform(df['purpose'].astype(str))
df['emp_length'] = le.fit_transform(df['emp_length'].astype(str))
df['home_ownership'] = le.fit_transform(df['home_ownership'].astype(str))
df['term'] = le.fit_transform(df['term'].astype(str))

In [25]:
df['loan_status'].value_counts()

Fully Paid                                             1076751
Charged Off                                             268559
Late (31-120 days)                                       21467
In Grace Period                                           8436
Late (16-30 days)                                         4349
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     40
Name: loan_status, dtype: int64

In [26]:
df['loan_status'] = df.loan_status.map({'Fully Paid':0, 
                                            'Charged Off': 1
                                            , 'Late (31-120 days)': 1
                                           , 'Late (16-30 days)': 1
                                           , 'In Grace Period': 0
                                           , 'Does not meet the credit policy. Status:Fully Paid': 0
                                           , 'Does not meet the credit policy. Status:Charged Off': 1
                                           , 'Default': 1})

In [27]:
df['loan_status'].value_counts()

0    1087175
1     295176
Name: loan_status, dtype: int64

In [28]:
path_write ="/Users/a06411/Documents/data_hub/lending_club/df_preprocessed_ld_0.pkl"

In [29]:
df.to_pickle(path_write)